In [ ]:
import colorcet as cc
import folium
import ee
ee.Authenticate()
ee.Initialize()

: 

In [3]:
# 画像取得したいポリゴン座標
coords = [
    [140.048947,35.14381],
    [140.048947,35.161424],
    [140.068946,35.161424],
    [140.068946,35.14381],
    [140.048947,35.14381]
]


In [4]:
# foliumマップにGEEを表示させる関数
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict["tile_fetcher"].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        overlay=True,
        control=True,
    ).add_to(self)


# 雲に覆われているピクセルの補正処理関数
def maskS2clouds(image):
    qa = image.select("QA60")
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11
    mask = qa.bitwiseAnd(cloudBitMask).eq(0) and qa.bitwiseAnd(cirrusBitMask).eq(0)
    return image.updateMask(mask).divide(10000)



In [5]:
# 対象期間（開始）
START_DATE ='2025-4-21'
# 対象期間（終了）
END_DATE = '2025-11-07'
# 雲被覆率フィルタリングの上限値
CLOUD_COVER_RATE = 30


In [6]:
# ImageCollectionの絞り込み
imgcol = (
  ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
  .filterBounds(ee.Geometry.Polygon(coords))
  .filterDate(ee.Date(START_DATE), ee.Date(END_DATE))
  .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE',CLOUD_COVER_RATE))
)

# ImageCollectionに格納されているImageIDのリスト化と確認
asset_id_list = imgcol.aggregate_array('system:id').getInfo()
print(asset_id_list)


['COPERNICUS/S2_SR_HARMONIZED/20250421T012659_20250421T012656_T54SVD', 'COPERNICUS/S2_SR_HARMONIZED/20250421T012659_20250421T012656_T54SVE', 'COPERNICUS/S2_SR_HARMONIZED/20250503T011711_20250503T011822_T54SVD', 'COPERNICUS/S2_SR_HARMONIZED/20250503T011711_20250503T011822_T54SVE', 'COPERNICUS/S2_SR_HARMONIZED/20250605T012721_20250605T012820_T54SVD', 'COPERNICUS/S2_SR_HARMONIZED/20250617T011649_20250617T011651_T54SVD', 'COPERNICUS/S2_SR_HARMONIZED/20250617T011649_20250617T011651_T54SVE', 'COPERNICUS/S2_SR_HARMONIZED/20250617T013111_20250617T013108_T54SVD', 'COPERNICUS/S2_SR_HARMONIZED/20250622T011711_20250622T011711_T54SVD', 'COPERNICUS/S2_SR_HARMONIZED/20250622T011711_20250622T011711_T54SVE', 'COPERNICUS/S2_SR_HARMONIZED/20250630T012659_20250630T012657_T54SVD', 'COPERNICUS/S2_SR_HARMONIZED/20250707T011659_20250707T011655_T54SVD', 'COPERNICUS/S2_SR_HARMONIZED/20250707T011659_20250707T011655_T54SVE', 'COPERNICUS/S2_SR_HARMONIZED/20250707T013111_20250707T013113_T54SVD', 'COPERNICUS/S2_SR_H

In [7]:
# 画像をGoogleDrive内に出力
for i,asset_id in enumerate(asset_id_list):
  task = ee.batch.Export.image.toDrive(
    image = ee.Image(asset_id_list[i]).select(['TCI_R','TCI_G','TCI_B']),
    region = ee.Geometry.Polygon(coords),
    description = 'Sample'+str(i),
  )
  task.start()


In [8]:
# NDVIの計算用関数
def calc_ndvi(image):
    return ee.Image(
        image.expression(
            "(NIR-RED)/(NIR+RED)",
            {"RED": image.select("B4"), "NIR": image.select("B8")},
        )
    ).rename("NDVI")


# Sentinel-2のデータの収集
Sentinel2 = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")

# 対象日付のリスト（各日付について前後数日の範囲で取得を試みる）
target_dates = [
    "2025-04-21",
    "2025-08-24",
    "2025-09-08",
    "2025-10-28",
    "2025-11-07"
]

# 各日付の画像を取得して結合
from datetime import datetime, timedelta

collections = []
date_info = []  # デバッグ用：各日付の取得状況を記録
date_images = {}  # 各日付の画像を個別に保存（Cell 7で使用するため）

for date in target_dates:
    # まず、対象日付の画像を取得
    date_collection_exact = (
        Sentinel2
        .filterDate(f"{date}T00:00:00", f"{date}T23:59:59")
        .filterBounds(ee.Geometry.Polygon(coords))
        .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 30))
        .map(maskS2clouds)
    )
    
    # 取得できた画像数を確認
    try:
        count_exact = date_collection_exact.size().getInfo()
        
        if count_exact > 0:
            # 対象日付の画像が見つかった場合
            date_collection = date_collection_exact
            date_info.append(f"{date}: {count_exact}枚取得（正確な日付）")
        else:
            # 対象日付の画像が見つからない場合、前後3日間の範囲で取得
            date_obj = datetime.strptime(date, "%Y-%m-%d")
            start_date = (date_obj - timedelta(days=3)).strftime("%Y-%m-%d")
            end_date = (date_obj + timedelta(days=3)).strftime("%Y-%m-%d")
            
            date_collection_range = (
                Sentinel2
                .filterDate(f"{start_date}T00:00:00", f"{end_date}T23:59:59")
                .filterBounds(ee.Geometry.Polygon(coords))
                .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 30))
                .map(maskS2clouds)
                .sort("system:time_start")
            )
            
            count_range = date_collection_range.size().getInfo()
            
            if count_range > 0:
                # 対象日付に最も近い1枚を取得
                target_time = ee.Date(f"{date}T12:00:00").millis()
                # 時間差を計算して最も近い画像を選択
                date_collection = date_collection_range.map(
                    lambda img: img.set("time_diff", ee.Number(img.get("system:time_start")).subtract(target_time).abs())
                ).sort("time_diff").limit(1)
                date_info.append(f"{date}: {count_range}枚取得（前後3日間の範囲、最も近い1枚を選択）")
            else:
                date_info.append(f"{date}: 0枚取得（前後3日間の範囲でも見つかりませんでした）")
                count_range = 0
            
            count_exact = count_range
        
    except Exception as e:
        date_info.append(f"{date}: エラー - {str(e)}")
        count_exact = 0
    
    if count_exact > 0:
        collections.append(date_collection)
        # 各日付の最初の画像を個別に保存（Cell 7で使用するため）
        date_image = date_collection.sort("system:time_start").first()
        date_images[date] = date_image
        # 各日付のNDVIも計算して保存（デバッグ用）
        date_ndvi = calc_ndvi(date_image)
        date_images[f"{date}_ndvi"] = date_ndvi

# デバッグ情報を表示
print("各日付の取得状況:")
for info in date_info:
    print(f"  {info}")

# すべてのコレクションを結合
if len(collections) == 0:
    raise ValueError("取得できた画像がありません。日付範囲やフィルター条件を確認してください。")
elif len(collections) == 1:
    term_selected = collections[0]
else:
    term_selected = ee.ImageCollection(collections[0])
    for col in collections[1:]:
        term_selected = term_selected.merge(col)

# NDVIの計算
ndvi_selected = term_selected.map(calc_ndvi)

# 各日付のNDVIを確認（デバッグ用）
print("\n各日付のNDVI画像の確認:")
for date in target_dates:
    if f"{date}_ndvi" in date_images:
        try:
            ndvi_img = date_images[f"{date}_ndvi"]
            # 画像のメタデータを確認
            time_start = ndvi_img.get("system:time_start").getInfo()
            if time_start:
                from datetime import datetime
                date_str = datetime.fromtimestamp(time_start / 1000).strftime("%Y-%m-%d %H:%M:%S")
                print(f"  {date}: NDVI画像あり (撮影日時: {date_str})")
            else:
                print(f"  {date}: NDVI画像あり (撮影日時: 不明)")
        except Exception as e:
            print(f"  {date}: NDVI画像の確認中にエラー - {str(e)}")
    else:
        print(f"  {date}: NDVI画像なし")

# NDVIの差を計算（最初と最後の日付の差）
# date_imagesから直接取得して確実に最初と最後の日付を使用
ndvi_first = date_images.get("2025-04-21_ndvi")
ndvi_last = date_images.get("2025-11-07_ndvi")

if ndvi_first is None or ndvi_last is None:
    # フォールバック: ndvi_selectedから取得
    ndvi_first = ndvi_selected.sort("system:time_start").first()
    ndvi_last = ndvi_selected.sort("system:time_start", False).first()
    print("警告: date_imagesから取得できなかったため、ndvi_selectedから取得しました。")
else:
    print("✓ 最初と最後のNDVI画像をdate_imagesから取得しました。")

# NDVIの差分計算を確認
print("\nNDVI差分計算の確認:")
print("  最初の日付: 2025-04-21")
print("  最後の日付: 2025-11-07")
print("  計算式: ndvi = ndvi_last.subtract(ndvi_first)")

# 差分を計算
ndvi = ndvi_last.subtract(ndvi_first)

# 差分が正しく計算されているか確認（サンプルポイントで検証）
try:
    # ポリゴンの中心点でNDVI値を取得
    center_point = ee.Geometry.Polygon(coords).centroid()
    
    # 最初と最後のNDVI値と差分を取得
    # バンド名を確認
    first_bands = ndvi_first.bandNames().getInfo()
    last_bands = ndvi_last.bandNames().getInfo()
    diff_bands = ndvi.bandNames().getInfo()
    print(f"  最初のNDVIバンド名: {first_bands}")
    print(f"  最後のNDVIバンド名: {last_bands}")
    print(f"  差分のNDVIバンド名: {diff_bands}")
    
    # バンド名を使用（"NDVI"または最初のバンド）
    first_band = "NDVI" if "NDVI" in first_bands else first_bands[0] if first_bands else "constant"
    last_band = "NDVI" if "NDVI" in last_bands else last_bands[0] if last_bands else "constant"
    diff_band = "NDVI" if "NDVI" in diff_bands else diff_bands[0] if diff_bands else "constant"
    
    first_ndvi_value = ndvi_first.select(first_band).reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=center_point,
        scale=10,
        maxPixels=1e9
    ).getInfo()
    
    last_ndvi_value = ndvi_last.select(last_band).reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=center_point,
        scale=10,
        maxPixels=1e9
    ).getInfo()
    
    diff_ndvi_value = ndvi.select(diff_band).reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=center_point,
        scale=10,
        maxPixels=1e9
    ).getInfo()
    
    # 値の取得（バンド名に対応）
    first_val = first_ndvi_value.get(first_band) if first_ndvi_value else "N/A"
    if first_val is None and first_ndvi_value:
        first_val = list(first_ndvi_value.values())[0] if first_ndvi_value else "N/A"
    
    last_val = last_ndvi_value.get(last_band) if last_ndvi_value else "N/A"
    if last_val is None and last_ndvi_value:
        last_val = list(last_ndvi_value.values())[0] if last_ndvi_value else "N/A"
    
    diff_val = diff_ndvi_value.get(diff_band) if diff_ndvi_value else "N/A"
    if diff_val is None and diff_ndvi_value:
        diff_val = list(diff_ndvi_value.values())[0] if diff_ndvi_value else "N/A"
    
    print(f"\n中心点でのNDVI値（検証用）:")
    print(f"  最初（2025-04-21）: {first_val}")
    print(f"  最後（2025-11-07）: {last_val}")
    print(f"  差分（計算値）: {diff_val}")
    
    # 手動計算で検証
    if isinstance(first_val, (int, float)) and isinstance(last_val, (int, float)) and isinstance(diff_val, (int, float)):
        manual_diff = last_val - first_val
        print(f"  手動計算（最後-最初）: {manual_diff}")
        if abs(diff_val - manual_diff) < 0.001:
            print("  ✓ 差分計算は正しく行われています。")
        else:
            print(f"  ⚠ 差分計算に不一致があります（差: {abs(diff_val - manual_diff)}）")
    else:
        print("  ⚠ 値の取得に失敗しました。")
        
except Exception as e:
    print(f"  検証中にエラー: {str(e)}")

print("\n✓ NDVIの差（最後-最初）を計算しました。")


各日付の取得状況:
  2025-04-21: 2枚取得（正確な日付）
  2025-08-24: 2枚取得（正確な日付）
  2025-09-08: 2枚取得（正確な日付）
  2025-10-28: 2枚取得（正確な日付）
  2025-11-07: 2枚取得（正確な日付）

各日付のNDVI画像の確認:
  2025-04-21: NDVI画像あり (撮影日時: 不明)
  2025-08-24: NDVI画像あり (撮影日時: 不明)
  2025-09-08: NDVI画像あり (撮影日時: 不明)
  2025-10-28: NDVI画像あり (撮影日時: 不明)
  2025-11-07: NDVI画像あり (撮影日時: 不明)
✓ 最初と最後のNDVI画像をdate_imagesから取得しました。

NDVI差分計算の確認:
  最初の日付: 2025-04-21
  最後の日付: 2025-11-07
  計算式: ndvi = ndvi_last.subtract(ndvi_first)
  最初のNDVIバンド名: ['NDVI']
  最後のNDVIバンド名: ['NDVI']
  差分のNDVIバンド名: ['NDVI']

中心点でのNDVI値（検証用）:
  最初（2025-04-21）: 0.841169853768279
  最後（2025-11-07）: 0.21478873239436616
  差分（計算値）: -0.6263811213739129
  手動計算（最後-最初）: -0.6263811213739129
  ✓ 差分計算は正しく行われています。

✓ NDVIの差（最後-最初）を計算しました。


In [9]:
# マップの中心に表示する緯度経度
lat, lon = 35.15573545801728, 140.05841343276924  # 緯度, 経度の順

# NDVI値の可視化パラメータ（0-1の範囲）
# 低NDVI（植生が少ない）= 白/赤、高NDVI（植生が多い）= 青/緑
visualization = {
    "min": 0.0, 
    "max": 1.0, 
    "palette": ["white", "red", "yellow", "lightgreen", "green", "blue"]
}

# NDVI変化の可視化パラメータ（負の値も含む：-1.0から1.0の範囲）
# 赤系（減少）から緑系（増加）へのパレットを使用
visualization_diff = {
    "min": -1.0, 
    "max": 1.0, 
    "palette": ["red", "orange", "yellow", "lightgreen", "green"]
}

# マップを表示した際の初期位置と拡大値の設定
# zoom_start: 10=市街地レベル, 12=地区レベル, 14=街区レベル, 16=建物レベル
my_map = folium.Map(location=[lat, lon], zoom_start=14)

# foliumマップにadd_ee_layer関数を追加
folium.Map.add_ee_layer = add_ee_layer

# 最初と最後の日付のNDVIを表示（date_imagesから直接取得）
ndvi_first_date = date_images.get("2025-04-21_ndvi")
ndvi_last_date = date_images.get("2025-11-07_ndvi")

# foliumマップに計算したNDVIを表示させる（最初、最後、変化のみ）
if ndvi_first_date:
    my_map.add_ee_layer(ndvi_first_date, visualization, "NDVI_最初（2025-04-21）")
if ndvi_last_date:
    my_map.add_ee_layer(ndvi_last_date, visualization, "NDVI_最後（2025-11-07）")
# NDVI変化は負の値も含むため、専用の可視化パラメータを使用
my_map.add_ee_layer(ndvi, visualization_diff, "NDVI_変化（最後-最初）")

# RGBの表示設定
visualization_RGB = {"min": 0.0, "max": 0.3, "bands": ["B4", "B3", "B2"]}

# 各日付のRGB画像を取得して表示
target_dates = [
    "2025-04-21",
    "2025-08-24",
    "2025-09-08",
    "2025-10-28",
    "2025-11-07"
]

# foliumマップに各日付のRGBを表示させる
# Cell 6で保存した各日付の画像を使用
for date in target_dates:
    if date in date_images:
        try:
            rgb_date = date_images[date]
            # 画像が有効かどうかを確認（bandNamesで確認）
            band_names = rgb_date.bandNames().getInfo()
            if len(band_names) > 0:
                # 日付の表示用（YYYY/MM/DD形式）
                date_display = date.replace("-", "/")
                my_map.add_ee_layer(rgb_date, visualization_RGB, f"RGB_{date_display}")
            else:
                print(f"警告: {date}の画像にバンドがありません。スキップします。")
        except Exception as e:
            print(f"警告: {date}の画像を追加できませんでした: {str(e)}")
    else:
        print(f"警告: {date}の画像が見つかりませんでした。スキップします。")

# 　複数のマップを重ね合わせる
my_map.add_child(folium.LayerControl(collapsed=False).add_to(my_map))


In [10]:
# マップの保存
my_map.save("ndvi_chiba_rev4.html")